In [1]:
%run Metric.ipynb
%run BreakSectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb


class BreakSectionAnalyser:
    cate_single_level_sections = 'single_level'
    cate_resonate_on_week_and_day = 'resonate_WD'
    
    def __init__(self,scale_start,scale_end,context):
        self.scale_start = scale_start
        self.scale_end = scale_end
        self.context = context
        self.short = context['short']
        self.mid = context['mid']
        
        self.symbols = get_symbol_slices(scale_start,scale_end,data_dir)

        
    def build_sections_file_path(self, cate,level):
        return beak_sections_of_multip_symbols_dir + cate +'_'+str(self.short) +'_'+str(self.mid) +'_'+level +'_'+str(self.scale_start)+'_'+str(self.scale_end)+ '_all_sections_' + break_section_version+'.txt'
        
    def get_all_sections_of_all_symbols(self,level):
        file_path = self.build_sections_file_path(self.cate_single_level_sections,level)
        
        all_sections_df = pd.DataFrame(columns=break_section_columns)

        if(os.path.exists(file_path)):
            all_sections_df = pd.read_csv(file_path)
        else:
            for i in range(0, len(self.symbols)):
                sectionManager = BreakSectionManager(self.symbols[i], self.context)
                temp_section = sectionManager.get_all_break_sections()

                all_sections_df = pd.concat([temp_section,all_sections_df], ignore_index=True, sort=False)

            all_sections_df.to_csv(file_path, index=0)
        
        print('get_all_sections_of_all_symbols done total:' + str(all_sections_df.shape[0]))
        return all_sections_df
    
    
    def build_resonate_section_on_week_and_day(self):
        resonate_df = pd.DataFrame(columns=break_resonate_section_columns)
        
        week_sections_df = self.get_all_sections_of_all_symbols(level_week)
        
        symbols_set = week_sections_df['symbol'].drop_duplicates()

        progress = 0
        total_count = symbols_set.shape[0]
        
        print('find all break sections...')
        for symbol in symbols_set:
            sectionManager = BreakSectionManager(symbol, level_day, self.short, self.mid)
            dates = week_sections_df[(week_sections_df['symbol']==symbol)]['d_s_date']

            progress += 1
            if(progress % 4 ==0):
                print(str(round(100*(progress/total_count),2))+'%', end=' ')
            
            for date in dates:
                w_section = week_sections_df[(week_sections_df['symbol']==symbol) & (week_sections_df['d_s_date']==date)]

                w_start_date = w_section['s_date'].values[0]
                w_end_date = w_section['e_date'].values[0]
                
                d_sections = sectionManager.find_sections_between(w_start_date, w_end_date)

                for index,row in d_sections.iterrows():
                    d_s_date = row['s_date']
                    d_e_date = row['e_date']
                    
                    d_s_price = row['s_price']
                    d_e_price = row['e_price']
                    

                    resonate_df.loc[resonate_df.shape[0]+1] = [symbol,w_start_date, w_end_date,
                                                               d_s_date,d_e_date,d_s_price,d_e_price,
                                                               d_ampl,d_boll,d_price_incre_per,
                                                               d_ma_short_direction,d_ma_mid_direction,
                                                               w_ma_short_direction,w_ma_mid_direction,
                                                               d_break_mid_ma_price_incre_per,
                                                               w_boll99,w_enter_to_highest_incre_per,
                                                               d_boll99,d_enter_to_highest_incre_per]

        return resonate_df
        